# 4. TimeSeries Example From DB Guide Webpage
 - 미국 LA시가 무료로 제공하는 데이터를 이용해보고자 한다. 
 - https://data.lacity.org/A-Safe-City/LAPD-Calls-for-Service-YTD-2014/mgue-vbsx
  - LA에서 일어난 경찰 신고 데이터가 담겨져 있다. 
 - JSON 형태로 데이터를 가지고 오겠다. 
  - https://data.lacity.org/resource/mgue-vbsx.json?group=date&call_type_code=507P&select=date_trunc_ymd(dispatch_date)%20AS%20date%2C%20count(*)&$order=date 
  - https://data.lacity.org/resource/mgue-vbsx.json 만 입력하면 모든 데이터를 가지고 올 수 있다.
  - key값을 이용하여 507P(파티의 씨끄러운 소리로 인한 신고)만 가지고 오겠다. 
 - pandas를 이용하여 데이터를 DataFrame 객체로 가져온 이후 이를 그래프로 표현할 때 Bokeh라는 것을 사용할 것이다. 이를 이용하여 html로도 만들 수 있다.

In [1]:
import numpy as np
import pandas as pd
import datetime
import urllib

In [5]:
from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict

In [6]:
query = ("https://data.lacity.org/resource/mgue-vbsx.json?"
         "$group=date"
"&call_type_code=507P"
"&$select=date_trunc_ymd(dispatch_date)%20AS%20date%2C%20count(*)"
"&$order=date")

In [8]:
query

'https://data.lacity.org/resource/mgue-vbsx.json?$group=date&call_type_code=507P&$select=date_trunc_ymd(dispatch_date)%20AS%20date%2C%20count(*)&$order=date'

In [9]:
raw_data = pd.read_json(query)

In [10]:
raw_data.head()

,count,date
0,285,2014-01-01
1,16,2014-01-02
2,33,2014-01-03
3,134,2014-01-04
4,101,2014-01-05


 - 첫 번째로는 컬럼을 삽입해보자. 즉 요일(day_of_week)과 주(week) 표시를 넣는 작업을 해보자.

In [11]:
raw_data['day_of_week'] = [date.dayofweek for date in raw_data["date"]]

In [12]:
raw_data['week'] = [(date - datetime.timedelta(days=date.dayofweek)).strftime("%Y-%m-%d") for date in raw_data["date"]]

In [13]:
raw_data.head()

,count,date,day_of_week,week
0,285,2014-01-01,2,2013-12-30
1,16,2014-01-02,3,2013-12-30
2,33,2014-01-03,4,2013-12-30
3,134,2014-01-04,5,2013-12-30
4,101,2014-01-05,6,2013-12-30


### Pivot_table

In [14]:
data = raw_data.pivot(index='week', columns='day_of_week', values='count')
data = data.fillna(0)

In [15]:
data.head()

day_of_week,0,1,2,3,4,5,6
week,,,,,,,
2013-12-30,0.0,0.0,285.0,16.0,33.0,134.0,101.0
2014-01-06,11.0,17.0,12.0,10.0,55.0,178.0,146.0
2014-01-13,13.0,11.0,13.0,17.0,50.0,233.0,174.0
2014-01-20,28.0,8.0,13.0,14.0,47.0,262.0,170.0
2014-01-27,24.0,13.0,14.0,19.0,41.0,181.0,131.0


In [16]:
weeks = list(data.index)
days = ["Mon", "Tues", "Wed", "Thurs", "Fri", "Sat", "Sun"]

In [18]:
max_count = raw_data["count"].max()
day_of_week = []
week = []
color = []
parties = []

In [20]:
list(enumerate(days))

[(0, 'Mon'),
 (1, 'Tues'),
 (2, 'Wed'),
 (3, 'Thurs'),
 (4, 'Fri'),
 (5, 'Sat'),
 (6, 'Sun')]

In [21]:
max_count

373

In [29]:
"%x"%16

'10'

In [30]:
for w in weeks:
    for idx, day in enumerate(days):
        day_of_week.append(day)
        week.append(w)
        count = data.loc[w][idx]
        parties.append(count)
        color.append("#%02x%02x%02x" % (255, int(255 - (count / max_count) * 255.0), int(255 - (count / max_count) * 255.0)))

In [31]:
data=dict(
        day_of_week=day_of_week,
        week=week,
        color=color,
        parties=parties,
    )

In [33]:
source = ColumnDataSource(data)

C:\Anaconda3\envs\work\lib\site-packages\bokeh\models\sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))


In [34]:
output_file('party_call_in_la.html')

In [35]:
TOOLS = "hover"

In [36]:
p = figure(title='파티의 소음으로 인한 신고 건수 현황 (LA)',
    x_range=weeks,
    y_range=list(reversed(days)),
    tools=TOOLS)
p.plot_width=900
p.plot_height = 400
p.toolbar_location='left'

In [37]:
p.rect("week", "day_of_week", 1, 1, source=source, color=color, line_color=None)

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "10pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3

hover = p.select(dict(type=HoverTool))
hover.tooltips = OrderedDict([
    ('count', '@parties'),
])

show(p)

C:\Anaconda3\envs\work\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
C:\Anaconda3\envs\work\lib\site-packages\bokeh\models\sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))


![img1](data/1.PNG)